In [1]:
import models.InceptionResNetV1_tf as Incep 
import tensorflow as tf

def load_pb_model(pb_file_path):
    with tf.io.gfile.GFile(pb_file_path, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
    return graph

def load_weights_from_pb(pb_file_path):
    graph = load_pb_model(pb_file_path)
    with tf.compat.v1.Session(graph=graph) as sess:
        weights = {v.name: v.eval(session=sess) for v in tf.compat.v1.global_variables()}
    return weights

pb_file_path = 'models/20180402-114759.pb'
weights = load_weights_from_pb(pb_file_path)


In [2]:
input_shape = (None, 160, 160, 3)
inputs = tf.keras.Input(shape=(160, 160, 3))
net = Incep.inception_resnet_v1(inputs, is_training=False)
facenet_model = tf.keras.Model(inputs=inputs, outputs=net)

for layer in facenet_model.layers:
    if layer.name in weights:
        layer.set_weights(weights[layer.name])

for layer in facenet_model.layers:
    layer.trainable = False


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'dataset/train_eq'
val_dir = 'dataset/val_eq'

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 48000 images belonging to 480 classes.
Found 0 images belonging to 60 classes.


In [5]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

x = facenet_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

fine_tuned_model = Model(inputs=facenet_model.input, outputs=predictions)
fine_tuned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# Callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
csv_logger = CSVLogger('training_log.csv', append=True)

# Train the model
history = fine_tuned_model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[checkpoint, csv_logger]
)

In [ ]:
# Save the fine-tuned model
fine_tuned_model.save('fine_tuned_facenets.h5')

# Evaluate the model
loss, accuracy = fine_tuned_model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# Callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
csv_logger = CSVLogger('training_log.csv', append=True)

# Train the model
history = fine_tuned_model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[checkpoint, csv_logger]
)